In [247]:
from tabulate import tabulate
import pandas as pd
import importlib
import util.Data as data_util
import util.DataframeProcess as df_proc
import json

In [248]:
importlib.reload(data_util)
importlib.reload(df_proc)

<module 'util.DataframeProcess' from 'D:\\scul\\uaic-ub\\licenta\\code\\util\\DataframeProcess.py'>

In [249]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_colwidth', 40)

# Data

In [250]:
csv_filename = r'../data_db/final_coordinates-conversion-v2.csv'
df_main = data_util.import_data(csv_filename)

In [251]:
df_data = df_main.copy()
df_data.shape

(1827, 26)

## all reports

## Focis number

In [252]:
#V1
df_data['counter'] = 1
ds_focis = df_data.groupby(['title', 'author'])['counter'].sum()

In [253]:
#V2
print("Count of Each group:")
df_group_focis = df_data.groupby(['title', 'author']
                                 ).size().reset_index(name="#Foci")
df_group_focis

Count of Each group:


,title,author,#Foci
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,13
1,A functional neuroimaging study of motivation and executive function,Taylor et al,41
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,21
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,58
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,10
...,...,...,...
56,Stimulus-dependent modulation of working memory for identity monitoring - A functional MRI study,Duggirala et al,62
57,The Architecture of Reward Value Coding in the Human Orbitofrontal Cortex,Sescousse et al,46
58,The neural system that bridges reward and cognition in humans An fMRI study,Pochon et al,43
59,Tracking the Hemodynamic Responses to Reward and Punishment in the Striatum,Delgado et al,32


## Subjects number

In [254]:
# group_subjects = df_data.groupby(['title','author']).agg({'subjects':['mean','min','max']})
# group_subjects = df_data.groupby(['title', 'author', 'subjects'])
# for key, item in group_subjects:
#     print(type(key), "\n\n")
#     break

# group_subjects.describe()

In [255]:
group_subjects = df_data.groupby(['title', 'author', 'subjects'])
dict_keys_subjects = group_subjects.groups.keys()
df_group_subjects = pd.DataFrame.from_records([*dict_keys_subjects], columns=['title', 'author', '#Subjects'])
df_group_subjects.head()

,title,author,#Subjects
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,31
1,A functional neuroimaging study of motivation and executive function,Taylor et al,12
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,12
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,14
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,14


In [256]:
df_final = df_group_focis.join(df_group_subjects[['title', '#Subjects']].set_index('title'), on='title')
df_final.head()

,title,author,#Foci,#Subjects
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,13,31
1,A functional neuroimaging study of motivation and executive function,Taylor et al,41,12
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,21,12
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,58,14
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,10,14


## Get set of contrast

In [257]:
# ds_group_contrast = df_data.groupby(['title', 'author'])['contrast'].apply(set)
# df_group_contrast = ds_group_contrast.to_frame().reset_index()
# df_group_contrast
# # df[['contrast']].iloc[0][0] <- set

In [258]:
# df_group_contrast = df_data.groupby(['title', 'author'])['contrast'].apply(set).reset_index(name='contrast_set')
df_group_contrast = df_data.groupby(['title', 'author'])['contrast'].apply(lambda x: set(x)).reset_index(
    name='contrast_set')
df_group_contrast.head(5)

,title,author,contrast_set
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,{2-back > 0-back}
1,A functional neuroimaging study of motivation and executive function,Taylor et al,{-}
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,"{reward > no reward, lose > no reward}"
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,{1-back > rest}
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,{high reward > low reward}


In [259]:
df_group_contrast[['contrast_set']].iloc[2][0]

{'lose > no reward', 'reward > no reward'}

In [260]:
df_final = df_final.join(df_group_contrast[['title', 'contrast_set']].set_index('title'), on='title')
df_final.head()

,title,author,#Foci,#Subjects,contrast_set
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,13,31,{2-back > 0-back}
1,A functional neuroimaging study of motivation and executive function,Taylor et al,41,12,{-}
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,21,12,"{reward > no reward, lose > no reward}"
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,58,14,{1-back > rest}
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,10,14,{high reward > low reward}


## Get set of keywords

In [261]:
#V1
# df_group_keywords = df_data.groupby(['title', 'author'])['keywords'].apply(set).reset_index(name='keywords_set')
df_group_keywords = df_data.groupby(['title', 'author'])['keywords'].apply(lambda x: set(x)).reset_index(
    name='keywords_set')
# ds_group_keywords = df_test.groupby(['title', 'author']).agg({"keywords": lambda x: set(x)})#%%

#df_group_keywords[['keywords_set']].iloc[0][0] <- set
df_group_keywords.head()

,title,author,keywords_set
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,{fMRI; working memory; talairach; n-back task; two-back task; visual letters}
1,A functional neuroimaging study of motivation and executive function,Taylor et al,"{fMRI; working memory; anticipation phase; monetary reward; MNI; n-back task; visual shape, fMRI; reward value; decision phase; monetary reward; MNI; n-back task; visual shape, fMRI; reward value; monetary reward; MNI; n-back task; visual shape, fMRI; reward value; anticipation phase; monetary reward; MNI; n-back task; visual shape, fMRI; working memory; decision phase; monetary reward; MNI; n-back task; visual shape}"
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,"{fMRI; reward value; anticipation phase; monetary reward; MID; talairach; performance-dependent; potential gain, fMRI; reward value; outcome phase; monetary reward; MID; talairach; performance-dependent; potential gain, fMRI; reward value; anticipation phase; monetary reward; MID; talairach; performance-dependent; potential loss}"
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,{fMRI; working memory; talairach; n-back task; one-back task; visual numbers}
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,"{fMRI; reward magnitude; reward value; outcome phase; decision making; monetary reward; high gain vs high loss; talairach; gambling;positive reward, fMRI; reward magnitude; reward value; outcome phase; decision making; monetary reward; high gain vs high loss; talairach; gambling; context-dependent reward;positive reward}"


In [262]:
df_group_keywords['keywords_split'] = df_group_keywords['keywords_set'].apply(df_proc.split_keys)
df_group_keywords.head()

,title,author,keywords_set,keywords_split
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,{fMRI; working memory; talairach; n-back task; two-back task; visual letters},"{fMRI, n-back task, two-back task, working memory, talairach, visual letters}"
1,A functional neuroimaging study of motivation and executive function,Taylor et al,"{fMRI; working memory; anticipation phase; monetary reward; MNI; n-back task; visual shape, fMRI; reward value; decision phase; monetary reward; MNI; n-back task; visual shape, fMRI; reward value; monetary reward; MNI; n-back task; visual shape, fMRI; reward value; anticipation phase; monetary reward; MNI; n-back task; visual shape, fMRI; working memory; decision phase; monetary reward; MNI; n-back task; visual shape}","{MNI, working memory, reward value, fMRI, n-back task, anticipation phase, monetary reward, visual shape, decision phase}"
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,"{fMRI; reward value; anticipation phase; monetary reward; MID; talairach; performance-dependent; potential gain, fMRI; reward value; outcome phase; monetary reward; MID; talairach; performance-dependent; potential gain, fMRI; reward value; anticipation phase; monetary reward; MID; talairach; performance-dependent; potential loss}","{MID, potential gain, performance-dependent, potential loss, reward value, outcome phase, fMRI, anticipation phase, monetary reward, talairach}"
3,Abnormal neural activity in partially remitted late-onset depression An fMRI study of one-back working memory task,Lee et al,{fMRI; working memory; talairach; n-back task; one-back task; visual numbers},"{fMRI, n-back task, working memory, talairach, one-back task, visual numbers}"
4,Activity in human reward-sensitive brain areas is strongly context dependent,Nieuwenhuis et al,"{fMRI; reward magnitude; reward value; outcome phase; decision making; monetary reward; high gain vs high loss; talairach; gambling;positive reward, fMRI; reward magnitude; reward value; outcome phase; decision making; monetary reward; high gain vs high loss; talairach; gambling; context-dependent reward;positive reward}","{positive reward, reward magnitude, fMRI, decision making, gambling, talairach, outcome phase, context-dependent reward, reward value, monetary reward, high gain vs high loss}"


In [263]:
df_group_keywords[['keywords_split']].iloc[0][0]

{'fMRI',
 'n-back task',
 'talairach',
 'two-back task',
 'visual letters',
 'working memory'}

In [264]:
df_final = df_final.join(df_group_keywords[['title', 'keywords_split']].set_index('title'), on='title')
df_final.head(3)

,title,author,#Foci,#Subjects,contrast_set,keywords_split
0,"A Rapid fMRI Task Battery for Mapping of Visual, Motor, Cognitive and Emotional Function",Drobyshevsky et al,13,31,{2-back > 0-back},"{fMRI, n-back task, two-back task, working memory, talairach, visual letters}"
1,A functional neuroimaging study of motivation and executive function,Taylor et al,41,12,{-},"{MNI, working memory, reward value, fMRI, n-back task, anticipation phase, monetary reward, visual shape, decision phase}"
2,A region of mesial prefrontal cortex tracks monetarily rewarding outcomes - characterization with rapid event-related fMRI,Knutson et al,21,12,"{reward > no reward, lose > no reward}","{MID, potential gain, performance-dependent, potential loss, reward value, outcome phase, fMRI, anticipation phase, monetary reward, talairach}"


In [265]:
df_final.shape

(61, 6)

## Print latex to text file

In [266]:
df_final_print = df_final.drop(['title'], axis=1)
df_final_print = df_final_print.sort_values(by=['author'])
# df_final_print = [df_final_print.iloc[:split_no, :], df_final_print.iloc[split_no:, :]]

In [267]:
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 40)
df_final_print.head()

,author,#Foci,#Subjects,contrast_set,keywords_split
33,Abler et al,43,15,"{risk > safe, high reward > low rewa...","{reward magnitude, modelling predict..."
12,Aharon et al,51,6,"{average female > average male, beau...","{fMRI, viewing, social reward, talai..."
15,Akitsuki et al,35,36,"{w4 > w1, w1 > l1, win modulation, a...","{positive reward, negative reward, g..."
6,Beneventi et al,24,12,{2-back > 1-back > 0-back},"{MNI, fMRI, n-back task, working mem..."
34,Carrion et al,34,13,"{3-back > 0-back, 2-back > 0-back}","{MNI, three-back task, working memor..."


In [268]:
df_final_print.rename(columns = {'author':'Author'}, inplace = True)
columns_to_print = ['Author', '#Foci', '#Subjects']
pd.set_option('display.max_colwidth', None)  # to show all the column values (-1)
latex_filepath = r'./latex/all_reports.txt'
with open(latex_filepath, 'w') as f:
    # for df in df_final_print:
    f.write(df_final_print[columns_to_print].to_latex(index=False))
pd.set_option('display.max_colwidth', 40)

C:\Users\Chh\AppData\Local\Temp\ipykernel_5092\1895782240.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_final_print[columns_to_print].to_latex(index=False))


In [269]:
print("Number of Subjects: ",df_final_print['#Subjects'].sum())
print("Number of Foci: ",df_final_print['#Foci'].sum())

Number of Subjects:  1238
Number of Foci:  1827


## Checks

### Contrast

In [270]:
list_of_contrast_set = list(df_final['contrast_set'])

In [271]:
set_all_contrasts = set()
for s in list_of_contrast_set:
    set_all_contrasts.update(s)
len(set_all_contrasts)

77

In [272]:
counter_contrast = data_util.df_counter(df_final.explode('contrast_set'), 'contrast_set',
                                        set_keywords=set_all_contrasts, order=1)
counter_contrast

{'>nan_values<': 0,
 'movement > no movement': 1,
 'beautiful female > beautiful male': 1,
 'punishment > reward': 1,
 'stroop block > baseline': 1,
 'beautiful male > average male': 1,
 'ar; sr > neutral activity': 1,
 'high reward > high loss': 1,
 'loss after winning streak > l1': 1,
 'disadvantageous > advantageous': 1,
 'lose > no reward': 1,
 'win after losing streak > w1': 1,
 'w4 > w1': 1,
 'expected reward > received reward and control > adhd': 1,
 '1-back > 2-back': 1,
 'win modulation': 1,
 'reward > negative feedback': 1,
 'monetary reward > erotic reward': 1,
 'beautiful faces > average faces': 1,
 'all w > all l': 1,
 'low-calorie and high-calorie': 1,
 'reward during run of success': 1,
 'decision-making > control task': 1,
 'risky gambles > safe gambles': 1,
 'task switching > control task': 1,
 'high-calorie > low-calorie': 1,
 'advantageous > disadvantageous': 1,
 'monetary reward magnitude': 1,
 'all female faces > all male faces': 1,
 'precue > retrocue': 1,
 'antic

### keywords

In [273]:
list_of_keywords_set = list(df_final['keywords_split'])

In [274]:
set_all_keywords = set()
for s in list_of_keywords_set:
    set_all_keywords.update(s)
len(set_all_keywords)

97

In [275]:
counter_keywords = data_util.df_counter(df_final.explode('keywords_split'), 'keywords_split',
                                        set_keywords=set_all_keywords, order=1)
counter_keywords

{'>nan_values<': 0,
 'potential loss': 1,
 'visual facial expression': 1,
 'verbal reward': 1,
 'increasing reward uncertainty': 1,
 'erotic reward': 1,
 'Stroop task': 1,
 'high calorie': 1,
 'perceptual matching task': 1,
 'visual shape': 1,
 'monetary and erotic reward': 1,
 'inhibition': 1,
 'effect of reward': 1,
 'negative expectancy': 1,
 'visual shape color': 1,
 'probability': 1,
 'dopamine': 1,
 'young': 1,
 'visual cue': 1,
 'extraversion': 1,
 'time effect': 1,
 'potential gain': 1,
 'conditioning task': 1,
 'time conditioning': 1,
 'visual identical shapes': 1,
 'high vs low calorie food': 1,
 'precue': 1,
 'increasing reward magnitude': 1,
 'positive expectancy': 1,
 'fractal image': 1,
 'reading': 1,
 'retrocue': 1,
 'decreasing activation': 1,
 'increasing activation': 1,
 'safe choice': 1,
 'adult': 1,
 'small incentive': 1,
 'two and four choice reversal learning task': 1,
 'response switching': 1,
 'visual spatial cue': 1,
 'visual color pies with probabilities': 1,


### Check contains/not contains

In [276]:
df_checks = df_final.copy()

In [277]:
############
check_set = {'high reward > low reward'}
mask = df_proc.check_column_elem(df=df_checks, colummn_name='contrast_set', check_set=check_set)
with pd.option_context('display.max_colwidth', None):  # to show all the column values
    print(tabulate(df_checks[mask], headers='keys'))

    title                                                                                                                                                            author                           #Foci    #Subjects  contrast_set                                                                                                                                                                                               keywords_split
--  ---------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------  -------  -----------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------

***

## Split wm and rest

In [278]:
df_data = df_main.copy()
mask_wm = df_data['keywords'].str.contains('working memory')

### Working Memory

In [279]:
df_wm = df_data[mask_wm]

In [280]:
df_wm['title'].unique().size

17

#### Focis number wm - for author,title and contrast

In [281]:
df_group_focis_wm = df_wm.groupby(['title', 'author', 'contrast']
                                  ).size().reset_index(name="#Foci")
df_group_focis_wm.head(1)

,title,author,contrast,#Foci
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,13


#### Subjects number wm

In [282]:
group_subjects_wm = df_wm.groupby(['title', 'author', 'subjects', 'contrast'])
dict_keys_subjects_wm = group_subjects_wm.groups.keys()
df_group_subjects_wm = pd.DataFrame.from_records([*dict_keys_subjects_wm],
                                                 columns=['title', 'author', '#Subjects', 'contrast'])
df_group_subjects_wm.head(10)

,title,author,#Subjects,contrast
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,31,2-back > 0-back
1,A functional neuroimaging study of m...,Taylor et al,12,-
2,Abnormal neural activity in partiall...,Lee et al,14,1-back > rest
3,Alterations in the functional neural...,Cruz et al,23,-
4,An fMRI study of working memory for ...,Beneventi et al,12,2-back > 1-back > 0-back
5,Anterior cingulate cortex differentl...,Di et al,49,1-back > 2-back
6,Anterior cingulate cortex differentl...,Di et al,49,2-back > 1-back
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,15,2-back > 0-back
8,Frontal Hypoactivation on Functional...,Carrion et al,13,2-back > 0-back
9,Frontal Hypoactivation on Functional...,Carrion et al,13,3-back > 0-back


In [283]:
df_final_wm = pd.merge(df_group_focis_wm, df_group_subjects_wm[['#Subjects']], left_index=True,
                       right_index=True)
df_final_wm.head(10)

,title,author,contrast,#Foci,#Subjects
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,13,31
1,A functional neuroimaging study of m...,Taylor et al,-,23,12
2,Abnormal neural activity in partiall...,Lee et al,1-back > rest,58,14
3,Alterations in the functional neural...,Cruz et al,-,27,23
4,An fMRI study of working memory for ...,Beneventi et al,2-back > 1-back > 0-back,24,12
5,Anterior cingulate cortex differentl...,Di et al,1-back > 2-back,13,49
6,Anterior cingulate cortex differentl...,Di et al,2-back > 1-back,13,49
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,2-back > 0-back,16,15
8,Frontal Hypoactivation on Functional...,Carrion et al,2-back > 0-back,16,13
9,Frontal Hypoactivation on Functional...,Carrion et al,3-back > 0-back,18,13


In [284]:
df_final_wm.shape

(24, 5)

## Get set of keywords

In [285]:
# get keywords and split them
df_group_keywords_wm = df_wm.groupby(['title', 'author', 'contrast'])['keywords'].apply(lambda x: set(x)).reset_index(
    name='keywords_set')
# df_group_keywords_wm.head(10)
df_group_keywords_wm['keywords_split'] = df_group_keywords_wm['keywords_set'].apply(df_proc.split_keys)

In [286]:
# list_key_set_wm = list(df_proc.list_of_sets_to_set(df_group_keywords_wm, 'keywords_split'))
# list_key_set_wm
set_key_set_wm_stimulus = {'visual word recognition',
                           # 'reading',
                           'verbal',
                           'verbal pseudo-words',
                           'visual numbers',
                           'visual shape',
                           'visual spatial cue',
                           'visual faces',
                           'visual letters',
                           'visual object',
                           'visual shape color',
                           'visual facial expression',
                           'visual identical shapes',
                           'visual scrambled drawings'}
set_key_set_wm_modality = {'reversal learning task',
                           # 'one-back task',
                           # 'two-back task',
                           # 'three-back task',
                           'n-back task',
                           'experimental task',
                           'perceptual matching task'}

In [287]:
# keep just the needed keywords
df_group_keywords_wm['Task Modality'] = df_group_keywords_wm['keywords_split'].apply(
    lambda x: ', '.join(x.intersection(set_key_set_wm_modality)))
df_group_keywords_wm['Task Stimulus'] = df_group_keywords_wm['keywords_split'].apply(
    lambda x: ', '.join(x.intersection(set_key_set_wm_stimulus)))

In [288]:
df_group_keywords_wm.head(8)

,title,author,contrast,keywords_set,keywords_split,Task Modality,Task Stimulus
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,{fMRI; working memory; talairach; n-...,"{fMRI, n-back task, two-back task, w...",n-back task,visual letters
1,A functional neuroimaging study of m...,Taylor et al,-,{fMRI; working memory; anticipation ...,"{MNI, working memory, fMRI, n-back t...",n-back task,visual shape
2,Abnormal neural activity in partiall...,Lee et al,1-back > rest,{fMRI; working memory; talairach; n-...,"{fMRI, n-back task, working memory, ...",n-back task,visual numbers
3,Alterations in the functional neural...,Cruz et al,-,{fMRI; working memory; decision maki...,"{MNI, fMRI, two and four choice reve...",reversal learning task,visual identical shapes
4,An fMRI study of working memory for ...,Beneventi et al,2-back > 1-back > 0-back,{fMRI; working memory; MNI; n-back t...,"{MNI, fMRI, n-back task, working mem...",n-back task,"visual facial expression, visual scr..."
5,Anterior cingulate cortex differentl...,Di et al,1-back > 2-back,{fMRI; working memory; MNI; n-back t...,"{MNI, fMRI, n-back task, working mem...",n-back task,visual letters
6,Anterior cingulate cortex differentl...,Di et al,2-back > 1-back,{fMRI; working memory; MNI; n-back t...,"{MNI, fMRI, n-back task, two-back ta...",n-back task,visual letters
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,2-back > 0-back,{fMRI; working memory; talairach; n-...,"{fMRI, n-back task, two-back task, v...",n-back task,verbal


In [289]:
df_final_wm = pd.merge(df_final_wm, df_group_keywords_wm[['Task Modality', 'Task Stimulus']], left_index=True,
                       right_index=True)

In [290]:
df_final_wm.head(11)

,title,author,contrast,#Foci,#Subjects,Task Modality,Task Stimulus
0,A Rapid fMRI Task Battery for Mappin...,Drobyshevsky et al,2-back > 0-back,13,31,n-back task,visual letters
1,A functional neuroimaging study of m...,Taylor et al,-,23,12,n-back task,visual shape
2,Abnormal neural activity in partiall...,Lee et al,1-back > rest,58,14,n-back task,visual numbers
3,Alterations in the functional neural...,Cruz et al,-,27,23,reversal learning task,visual identical shapes
4,An fMRI study of working memory for ...,Beneventi et al,2-back > 1-back > 0-back,24,12,n-back task,"visual facial expression, visual scr..."
5,Anterior cingulate cortex differentl...,Di et al,1-back > 2-back,13,49,n-back task,visual letters
6,Anterior cingulate cortex differentl...,Di et al,2-back > 1-back,13,49,n-back task,visual letters
7,Does caffeine modulate verbal workin...,Koppelstaetter et al,2-back > 0-back,16,15,n-back task,verbal
8,Frontal Hypoactivation on Functional...,Carrion et al,2-back > 0-back,16,13,n-back task,visual numbers
9,Frontal Hypoactivation on Functional...,Carrion et al,3-back > 0-back,18,13,n-back task,visual numbers


#### Print latex to text file - wm

In [291]:
df_final_print_wm = df_final_wm.drop(['title'], axis=1)
df_final_print_wm = df_final_print_wm.sort_values(by=['author'])
df_final_print_wm.rename(columns = {'author':'Author','contrast':'Contrast'}, inplace = True)
df_final_print_wm = df_final_print_wm[['Author', '#Foci', '#Subjects', 'Contrast', 'Task Modality', 'Task Stimulus']]
df_final_print_wm.head(2)

,Author,#Foci,#Subjects,Contrast,Task Modality,Task Stimulus
4,Beneventi et al,24,12,2-back > 1-back > 0-back,n-back task,"visual facial expression, visual scr..."
8,Carrion et al,16,13,2-back > 0-back,n-back task,visual numbers


In [292]:
pd.set_option('display.max_colwidth', None)  # to show all the column values (-1)
latex_filepath = r'./latex/wm_reports.txt'
with open(latex_filepath, 'w') as f:
    f.write(df_final_print_wm.to_latex(index=False))
pd.set_option('display.max_colwidth', 40)

C:\Users\Chh\AppData\Local\Temp\ipykernel_5092\1130413674.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df_final_print_wm.to_latex(index=False))


### Reward magnitude and type

In [293]:
df_reward = df_data[~mask_wm]

In [294]:
df_reward['title'].unique().size

46

In [295]:
df_reward.shape

(1330, 26)

#### Get set of keywords

In [296]:
# get keywords and split them
df_reward['keywords_split'] = df_reward['keywords'].apply(lambda x: df_proc.split_keys([x]))
df_reward.head(2)

C:\Users\Chh\AppData\Local\Temp\ipykernel_5092\3979881578.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reward['keywords_split'] = df_reward['keywords'].apply(lambda x: df_proc.split_keys([x]))


,importance,author,title,table_name,contrast,keywords,Name,Left/Right,Broadman Area,X(R),...,NV,index_col,MNIX,MNIY,MNIZ,TALX,TALY,TALZ,significance,keywords_split
0,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,mofc,l,11,-6,...,NaN,0,-6.0,24.0,-21.0,-6.436,22.720,-13.440,0.000025,"{MNI, reversal learning, fMRI, reinf..."
1,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,vmpfc,r,11,6,...,NaN,1,6.0,30.0,-9.0,4.724,27.408,-2.105,0.000050,"{MNI, reversal learning, fMRI, reinf..."


#### Create new columns

In [297]:
set_reward_type = {
    'erotic reward',
    'verbal reward',
    'monetary and erotic reward',
    'feedback reward',
    'taste reward',
    'context-dependent reward',  # maybe get it out
    'social reward',
    'food photo reward',
    'monetary reward'}

set_reward_contingency = {
    # 'viewing',  ## should be where is passive
    'passive',
    'gambling',
    'performance-dependent',
    'reversal learning',
    'n-back task'  # check this
}

set_reward_phase = {
    'anticipation phase',
    'decision phase',
    'outcome phase',
}

set_reward_decision_making = {
    'decision making'
}

In [298]:
set_reward_influence = {
    'negative reward',
    'positive reward',
}
set_keywords_others = {
    'safe choice',
    'risk choice',
    'positive expectancy',
    'negative expectancy',
}

In [299]:
df = pd.DataFrame()
list_new_cols = ['Reward Type', 'Reward Contingency', 'Reward Phase']
list_of_keys = [set_reward_type, set_reward_contingency, set_reward_phase]
for col_name, set_el in zip(list_new_cols, list_of_keys):
    df[col_name] = df_reward['keywords_split'].apply(lambda x: ', '.join(x.intersection(set_el)))

In [300]:
df['Decision Making'] = df_reward['keywords_split'].apply(
    lambda x: 'Choice' if set_reward_decision_making.issubset(x) else 'No Choice')

In [301]:
df.set_index(df_reward.index)
df_reward_final = pd.merge(df_reward, df, left_index=True,
                           right_index=True)
df_reward_final.head(2)

,importance,author,title,table_name,contrast,keywords,Name,Left/Right,Broadman Area,X(R),...,MNIZ,TALX,TALY,TALZ,significance,keywords_split,Reward Type,Reward Contingency,Reward Phase,Decision Making
0,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,mofc,l,11,-6,...,-21.0,-6.436,22.720,-13.440,0.000025,"{MNI, reversal learning, fMRI, reinf...",monetary reward,reversal learning,outcome phase,Choice
1,++,Glascher et al,Determining a Role for Ventromedial ...,Table 1-1 Coordinates and significan...,ar; sr > neutral activity,fMRI; decision making; outcome phase...,vmpfc,r,11,6,...,-9.0,4.724,27.408,-2.105,0.000050,"{MNI, reversal learning, fMRI, reinf...",monetary reward,reversal learning,outcome phase,Choice


In [302]:
## Check the new created keywords
for col_name in list_new_cols:
    counter_keywords_reward_type = data_util.df_counter(df=df_reward_final, column=col_name, order=1)
    print(col_name)
    print(json.dumps(counter_keywords_reward_type, indent=2, default=str))

Reward Type
{
  ">nan_values<": 0,
  "verbal reward, monetary reward": "18",
  "social reward, monetary reward": "19",
  "feedback reward": "20",
  "erotic reward": "20",
  "monetary and erotic reward": "20",
  "": "28",
  "context-dependent reward, monetary reward": "34",
  "food photo reward": "42",
  "taste reward": "51",
  "social reward": "155",
  "monetary reward": "923"
}
Reward Contingency
{
  ">nan_values<": 0,
  "reversal learning": "32",
  "n-back task": "46",
  "": "72",
  "passive": "210",
  "gambling": "449",
  "performance-dependent": "521"
}
Reward Phase
{
  ">nan_values<": 0,
  "outcome phase, anticipation phase": "21",
  "": "51",
  "decision phase": "62",
  "anticipation phase": "356",
  "outcome phase": "840"
}


In [303]:
counter_keywords_reward_type = data_util.df_counter(df=df_reward_final, column='Decision Making', order=1)
counter_keywords_reward_type

{'>nan_values<': 0, 'No Choice': 529, 'Choice': 801}

In [304]:
df = df_reward_final.groupby(['title', 'author', "Reward Contingency"]).size().reset_index(name="#Foci")
df.sort_values(by=['author'])

,title,author,Reward Contingency,#Foci
28,From uncertainty to reward - BOLD ch...,Abler et al,performance-dependent,43
7,Beautiful Faces Have Variable Reward...,Aharon et al,passive,51
10,Context-dependent cortical activatio...,Akitsuki et al,gambling,35
41,Right Ventromedial and Dorsolateral ...,Christakou et al,gambling,13
6,Are Attractive People Rewarding - Se...,Cloutier et al,passive,19
30,Individual differences in extraversi...,Cohen et al,performance-dependent,39
35,Learning to Like A Role for Human O...,Cox et al,gambling,28
27,Effort-Based Cost Benefit Valuation ...,Croxson et al,,9
26,Dorsal striatum responses to reward ...,Delgado et al,gambling,12
46,Tracking the Hemodynamic Responses t...,Delgado et al,gambling,32


In [305]:
df = df_reward_final.groupby(['title', 'author', "Reward Phase"]).size().reset_index(name="#Foci")
df.sort_values(by=['author'])

,title,author,Reward Phase,#Foci
38,From uncertainty to reward - BOLD ch...,Abler et al,"outcome phase, anticipation phase",21
37,From uncertainty to reward - BOLD ch...,Abler et al,outcome phase,5
36,From uncertainty to reward - BOLD ch...,Abler et al,anticipation phase,17
10,Beautiful Faces Have Variable Reward...,Aharon et al,outcome phase,51
13,Context-dependent cortical activatio...,Akitsuki et al,outcome phase,35
54,Right Ventromedial and Dorsolateral ...,Christakou et al,outcome phase,13
9,Are Attractive People Rewarding - Se...,Cloutier et al,outcome phase,19
41,Individual differences in extraversi...,Cohen et al,outcome phase,25
40,Individual differences in extraversi...,Cohen et al,anticipation phase,14
47,Learning to Like A Role for Human O...,Cox et al,outcome phase,28


In [306]:
df = df_reward_final.groupby(['title', 'author', "Reward Phase", "Reward Contingency"]).size().reset_index(name="#Foci")
df.sort_values(by=['author'])

,title,author,Reward Phase,Reward Contingency,#Foci
39,From uncertainty to reward - BOLD ch...,Abler et al,"outcome phase, anticipation phase",performance-dependent,21
38,From uncertainty to reward - BOLD ch...,Abler et al,outcome phase,performance-dependent,5
37,From uncertainty to reward - BOLD ch...,Abler et al,anticipation phase,performance-dependent,17
10,Beautiful Faces Have Variable Reward...,Aharon et al,outcome phase,passive,51
13,Context-dependent cortical activatio...,Akitsuki et al,outcome phase,gambling,35
...,...,...,...,...,...
54,Reward Value Coding Distinct From Ri...,Tobler et al,anticipation phase,performance-dependent,19
24,Dissociating contributions of ACC an...,Vassena et al,decision phase,gambling,13
25,Dissociating contributions of ACC an...,Vassena et al,outcome phase,gambling,65
12,Brain systems for assessing facial a...,Winston et al,outcome phase,passive,15


In [307]:
df_final_print_reward = df_reward_final.groupby(['title', 'author', 'subjects', 'Reward Phase', 'Reward Contingency','Reward Type']).size().reset_index(name="#Foci")

#### Print latex to text file - reward

In [308]:
#rename specific column names
df_final_print_reward.rename(columns = {'subjects':'#Subjects','title':'Title','author':'Author'}, inplace = True)
df_final_print_reward = df_final_print_reward[['Author', '#Foci', '#Subjects', 'Reward Phase', 'Reward Contingency', 'Reward Type']]
df_final_print_reward = df_final_print_reward.sort_values(by=['Author']).reset_index(drop=True)

In [309]:
df_final_print_reward.head(4)

,Author,#Foci,#Subjects,Reward Phase,Reward Contingency,Reward Type
0,Abler et al,17,15,anticipation phase,performance-dependent,monetary reward
1,Abler et al,5,15,outcome phase,performance-dependent,monetary reward
2,Abler et al,21,15,"outcome phase, anticipation phase",performance-dependent,monetary reward
3,Aharon et al,51,6,outcome phase,passive,social reward


In [310]:
df_final_print_reward.shape

(72, 6)

In [311]:
# pd.set_option('display.max_colwidth', None)  # to show all the column values (-1)
# latex_filepath = r'../data_db/latex/reward_reports.txt'
# with open(latex_filepath, 'w') as f:
#     f.write(df_final_print_reward.to_latex(index=False))
# pd.set_option('display.max_colwidth', 40)


In [312]:
split_no = 40
df_final_print_reward_list = [df_final_print_reward.iloc[:split_no, :], df_final_print_reward.iloc[split_no:, :]]

In [313]:
pd.set_option('display.max_colwidth', None)  # to show all the column values (-1)
latex_filepath = r'./latex/reward_reports.txt'
with open(latex_filepath, 'w') as f:
    for df in df_final_print_reward_list:
        f.write(df.to_latex(index=False))
pd.set_option('display.max_colwidth', 40)

C:\Users\Chh\AppData\Local\Temp\ipykernel_5092\1585037006.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(df.to_latex(index=False))
